# Statistical Learning Project

We need to categorize th problem. With the data we have, in this particular contexte, we want to classify in a book is rather a fiction or not.

Here we have the analysing task. We don't need to pre-process (note that much I mean)

Later, it will be interesting to transform the raw data with futher features (or transformation on the variables).

We also need to adress all if this points:

- The accuracy of the model. (we need to be careful with this)
- The interpretability of the model.
- The complexity of the model.
- The scalability of the model.
- How long does it take to build, train, and test the model?
- How long does it take to make predictions using the model?
- Does the model meet the business goal ?

And also: Optimize hyperparameters with (grid search, random search or Bayesian optimisation)

In [ ]:
"""IMPORT OF PCK"""

import os
import glob
import tqdm
import re

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)


import pandas as pd
import numpy as np
from scipy import stats
import math
import scipy.signal as signal
import random

from pathlib import Path
from IPython.core.display import display, HTML #display(HTML(data.to_html())) to show all variables
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

In [ ]:
"""DIRECTORY"""
os.getcwd()

## 1/ Import of data

In [ ]:
data_folder = Path(r"C:\Users\ville\PycharmProjects\stat_projetcs\git_project_stat\Stats_projetcs\Appr_Stat\data\Books_Dataset_20210108\archive")
bestsellers = data_folder / "bestsellers_with_categories.csv"
dta = pd.read_csv(bestsellers)

## 2/ Data Wrangling

In [ ]:
#Corrections for modelisation steps

dta["Genre"] = dta.Genre.replace(to_replace=['Fiction', 'Non Fiction'], value=[0, 1])
dta = dta.rename(columns={"User Rating": "Rating"})
dta.info()

In [ ]:
print(dta.head(5))
print(dta.tail(5))

<b>
<b>

We can see from the tail of our dataset that there are the same book, a multiple time.

The only difference is the year, in term of features. We can imagine that it represent the year when the book pop_up or the year when it becomes a bestsellers.
After a quick verification on internet, we can see that it represents: The year when the book was past of bestsellings.
Lets create a feature representing the time the book had been a bestsellings.

In the next steps, we will treat this problematic.

## -- Cleaning of Name column
Due to case sensitivity, the same book, with a different writing, can be misinterpreted to a different one.
We can correct this errors of case sensitivity by putting all the name in upper case.

In [ ]:
dta["Name"] = dta["Name"].str.upper()
#problem solved

## -- Cleaning of Author column

We identified some difference for the reference of some authors, such as J.K Rowling. To fix this error, we smooth out the author name to J.K Rowling (without space between ".")

In [ ]:
#To correct the space problem for few authors
for index, value in dta["Author"].items():
    if value.startswith("J."):
        change = value.replace(" ","")
        dta.loc[index, "Author"] = change
    else:
        pass


Creation of a new column which corresponds to the sum of year being best-sellings in total for the previous ten years.
Next we show the graph corresponding

In [ ]:
dta["sum_best"]=""
dta["sum_best"] = dta.groupby("Name").transform('count')

In [ ]:
plt.figure(figsize=(10,6))

ax = sns.countplot(data=dta, x=dta.Name.value_counts(), palette="Set2");

for p in ax.patches:
    ax.annotate(format(p.get_height(), 'd'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')

ax.set(xlabel="Appearances", ylabel="Count");
ax.set_title("Times a Title Shows up on Top 50 Charts (2009-2019)");
plt.show();

In [ ]:
dta_repeat = pd.DataFrame([x, y] for x, y in dta.Author.value_counts().iteritems() if (y > 1))
dta_repeat.columns = ['Author','Top 50 Appearances']
dta_repeat.columns

In [ ]:
plt.figure(figsize=(10,6))

ax = sns.countplot(data=dta_repeat, x="Top 50 Appearances", palette="Set2");

for p in ax.patches:
    ax.annotate('{:.1%}'.format(p.get_height()/len(dta_repeat)),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')

ax.set(xlabel="Appearances", ylabel="Count");
ax.set_title("Repeat Top 50 Appearances");
plt.show();


In [ ]:
#Looking for redundancy in the authors

plt.figure(figsize=(10,6))

ax = sns.countplot(data=dta, x=dta.Author.value_counts(), palette="Set2")

for p in ax.patches:
    ax.annotate(format(p.get_height(), 'd'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 9),
                   textcoords = 'offset points')

ax.set(xlabel="Appearances", ylabel="Number of Authors")
ax.set_title("Total Appearances in the Top 50 Charts")
plt.show()

## -- Basic statistics

In [ ]:
dta.describe().transpose()

## -- Look for correlation

In [ ]:
dta.shape

numerical = list(set(dta.columns) -
                 set(['Name','Author']))
corr_matrix = dta[numerical].corr()

sns.heatmap(corr_matrix, annot=True);
plt.xticks(rotation=45);

print(corr_matrix)

## -- Add of a weighted review column

In [ ]:
#Give an information on the dataset with the cosidered "best movies", with minimum 3000 review
data_order = dta.groupby(['Name', 'Author', 'Genre'], as_index=False)[['Rating', 'Reviews']].mean()
data_order = data_order[data_order['Reviews']>3000]
data_order = data_order.sort_values('Rating', ascending=False).head(20)
data_order


Use of weighted mean to attribute a review/rating which corresponds more to the reality field

with W_mean = ( ( v / m+v) * R) (( m / m+v)*C)

m = number of rating

C = mean Review

v= number of review

R = average numb of rating


In [ ]:
# Weighted rating
m = min(dta['Reviews'])
C = dta['Rating'].mean()
def weighted_rating(x, m=m, C=C):
    v = x['Reviews']
    R = x['Rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
dta['Weighted Rating'] = dta.apply(weighted_rating, axis=1)

In [ ]:
dta.groupby(['Name','Author','Genre'], as_index=False)[['Rating', 'Reviews', 'Weighted Rating']].mean().sort_values(by='Weighted Rating', ascending=False).head(10)

In [ ]:
dta.shape

numerical = list(set(dta.columns) -
                 set(['Name','Author']))
corr_matrix = dta[numerical].corr()

sns.heatmap(corr_matrix, annot=True);
plt.xticks(rotation=45);

print(corr_matrix)

In [ ]:
dta = dta.drop("Rating", axis=1)
dta.head(5)

## -- Step Outlier detection

## Visualisation des données pour la detection d'outlier
### Diagrammes à moustache des variables quantitatives


#Boxplot pour les variables quatitatives

In [ ]:
dta_quanti = dta[dta.columns.difference(["Name", "Author"])]
c = 'steelblue'
blue_dict =  {'patch_artist': True,
             'boxprops': dict(color=c, facecolor='w'),
             'capprops': dict(color=c),
             'flierprops': dict(color=c, markeredgecolor=c,marker="."),
             'medianprops': dict(color=c),
             'whiskerprops': dict(color=c)}

plt.close()
plt.figure(figsize=(15,10))
for i in np.arange(start=0,stop=dta_quanti.shape[1]):
    plt.subplot(4, 4, i+1)
    plt.boxplot(dta_quanti.iloc[:,i],**blue_dict)
    plt.xlabel(dta_quanti.columns[i])
plt.tight_layout()
plt.show()

## -- Step Bivariate analysis

In [ ]:
#With genre 0 = Fiction, genre 1 = Non fiction
sns.pairplot(dta, hue='Genre',vars=['Reviews', 'Price', 'Year',
       'sum_best', 'Weighted Rating'],markers='o')

In [ ]:
pairgrid = sns.PairGrid(dta,vars=['Reviews', 'Price', 'Year',
       'sum_best', 'Weighted Rating'], size = 4)
pairgrid = pairgrid.map_upper(plt.scatter, color = 'red', edgecolor = 'black', alpha = 0.2, s = 10)
pairgrid = pairgrid.map_diag(plt.hist, bins = 20, color = 'red',alpha = 0.2, edgecolor = 'k')# Map a density plot to the lower triangle
pairgrid = pairgrid.map_lower(sns.kdeplot, cmap = plt.cm.inferno,
                              shade = False, shade_lowest = False, alpha = 1.0, n_levels = 10)
pairgrid.add_legend()
plt.subplots_adjust(left=0.0, bottom=0.0, right=0.6, top=0.6, wspace=0.2, hspace=0.2)

In [ ]:


# Modelisation

## -- List of model we will use for our binairy classification

Note that we have to be careful with the rzegulaization step with the algorithm based on euclidian distance.

- Logistic Regression.
- k-Nearest Neighbors.
- Decision Trees.
- Support Vector Machine.
- Naive Bayes.

In [ ]:
#impot of pck for modelisation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from warnings import simplefilter
from sklearn import model_selection
from sklearn.preprocessing import  StandardScaler
from multiprocessing import Pool
import os
import warnings
from sklearn.exceptions import DataConversionWarning

In [ ]:
# definition of train: + train test slpit of data

seed = 7
validation_size = 0.5
scoring="accuracy"
#shuffle of the data
model_dta = dta_quanti.sample(frac=1).reset_index(drop=True)

X = model_dta.loc[:, model_dta.columns != 'Genre'].to_numpy().astype("int8")
y = model_dta.loc[:,"Genre"].to_numpy().astype("int8")

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)


Note for now we execute a train test split of 80/20, in the next steps, we will rafinate this part.

In [ ]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('RDF', RandomForestClassifier(n_estimators=100)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVC', SVC(gamma='auto')))
models.append(('GauNB', GaussianNB()))

results=[]
names=[]

In [ ]:
Lname=[models[0][0], models[1][0], models[2][0], models[3][0], models[4][0]]
Lmodel=[models[0][1], models[1][1], models[2][1], models[3][1], models[4][1]]

In [ ]:
def prediction(N,M):
#for name,model in models:
    from warnings import simplefilter
    simplefilter(action='ignore', category=FutureWarning)

    kfold=model_selection.KFold(n_splits=10,random_state=seed)
    cv_results=model_selection.cross_val_score(M,X_train,y_train,cv=10,scoring=scoring)
    results.append(cv_results)
    names.append(N)
    msg = "%s: %f (%f)" % (N, cv_results.mean(), cv_results.std())
    print(msg)
    return

In [ ]:
Lgt_accuracy = prediction(models[0][0],models[0][1])
Rdf_accuracy =  prediction(models[1][0],models[1][1])
KNN_accuracy=   prediction(models[2][0],models[2][1])
SVC_accuracy =  prediction(models[3][0],models[3][1])
GauNB_accuracy=   prediction(models[4][0],models[4][1])

# Hyperparameters optimization
